# Variational AutoEncoder

**Author:** [fchollet](https://twitter.com/fchollet)<br>
**Date created:** 2020/05/03<br>
**Last modified:** 2024/04/24<br>
**Description:** Convolutional Variational AutoEncoder (VAE) trained on MNIST digits.

## Setup

In [1]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pathlib
# import pandas as pd
import keras
from keras import layers, ops
from datetime import date

2024-11-21 15:16:30.184558: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Get the data

In [2]:
data_root = keras.utils.get_file(
    origin="https://www.kaggle.com/api/v1/datasets/download/jacksoncrow/stock-market-dataset",
    cache_dir='.', cache_subdir='data',
    extract=True)

In [3]:
tickers = np.load("tickers.npy")

In [4]:
data_root_path = pathlib.Path(data_root)
stock_dir = data_root_path.joinpath("stocks")
data_strs = [str(x) for x in stock_dir.iterdir() if x.stem in tickers]
data_columns = ["Open", "High", "Low", "Close", "Adj Close", "Volume"]

In [5]:
dataloader = tf.data.experimental.make_csv_dataset(
    column_defaults=["float32" for _ in range(6)],
    file_pattern=data_strs, batch_size=16, num_epochs=1, select_columns=data_columns
)#.map(lambda x: x | {"Volume": ops.cast(x["Volume"], dtype="float32")})

2024-11-21 15:16:45.292181: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-11-21 15:16:45.292226: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: legionix
2024-11-21 15:16:45.292239: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: legionix
2024-11-21 15:16:45.292376: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 560.35.3
2024-11-21 15:16:45.292402: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 560.35.3
2024-11-21 15:16:45.292412: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 560.35.3


## Create a sampling layer

In [6]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def __init__(self, seed=None, **kwargs):
        super().__init__(**kwargs)
        self.seed_generator = keras.random.SeedGenerator(seed=seed)

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = ops.shape(z_mean)[0]
        dim = ops.shape(z_mean)[1]
        epsilon = keras.random.normal(shape=(batch, dim), seed=self.seed_generator)
        return z_mean + ops.exp(0.5 * z_log_var) * epsilon

## Build the encoder

In [7]:
# https://stackoverflow.com/a/65246730
input_list = []
for column in data_columns:
    _input = keras.layers.Input(shape=(1,), name=column, dtype="float32")
    input_list.append(_input)

In [8]:
latent_dim = 2

encoder_inputs = layers.Concatenate(name="concat", trainable=False)(input_list)
x = layers.Dense(5, activation="relu")(encoder_inputs)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling(seed=1337)([z_mean, z_log_var])
encoder = keras.Model(input_list, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Open (InputLayer)   │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ High (InputLayer)   │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Low (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Close (InputLayer)  │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Adj Close           │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Volume (InputLayer) │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concat              │ (None, 6)         │          0 │ Open[0][0],       │
│ (Concatenate)       │                   │            │ High[0][0],       │
│                     │                   │            │ Low[0][0],        │
│                     │                   │            │ Close[0][0], Adj  │
│                     │                   │            │ Close[0][0],      │
│                     │                   │            │ Volume[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 5)         │         35 │ concat[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ z_mean (Dense)      │ (None, 2)         │         12 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ z_log_var (Dense)   │ (None, 2)         │         12 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sampling (Sampling) │ (None, 2)         │          0 │ z_mean[0][0],     │
│                     │                   │            │ z_log_var[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 59 (236.00 B)

 Trainable params: 59 (236.00 B)

 Non-trainable params: 0 (0.00 B)

## Build the decoder

In [9]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(5, activation="relu")(latent_inputs)
decoder_outputs = layers.Dense(6, activation="sigmoid")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 2)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │            15 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 6)              │            36 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 51 (204.00 B)

 Trainable params: 51 (204.00 B)

 Non-trainable params: 0 (0.00 B)

## Define the VAE as a `Model` with a custom `train_step`

In [10]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        _goodshape = tf.stack(list(data.values()), axis=1)
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = ops.mean(
                ops.sum(
                    keras.losses.binary_crossentropy(_goodshape, reconstruction),
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - ops.square(z_mean) - ops.exp(z_log_var))
            kl_loss = ops.mean(ops.sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

## Train the VAE

In [11]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())

In [12]:
dataloader

<_PrefetchDataset element_spec=OrderedDict([('Open', TensorSpec(shape=(None,), dtype=tf.float32, name=None)), ('High', TensorSpec(shape=(None,), dtype=tf.float32, name=None)), ('Low', TensorSpec(shape=(None,), dtype=tf.float32, name=None)), ('Close', TensorSpec(shape=(None,), dtype=tf.float32, name=None)), ('Adj Close', TensorSpec(shape=(None,), dtype=tf.float32, name=None)), ('Volume', TensorSpec(shape=(None,), dtype=tf.float32, name=None))])>

In [13]:
history = vae.fit(dataloader, epochs=30, batch_size=128)

Epoch 1/30


/nix/store/1hzi3scdwn996mwyf66vvww6j0p9yj6p-python3-3.11.10-env/lib/python3.11/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']. Received: the structure of inputs=OrderedDict([('Open', '*'), ('High', '*'), ('Low', '*'), ('Close', '*'), ('Adj Close', '*'), ('Volume', '*')])
  warnings.warn(


  26269/Unknown 40s 1ms/step - kl_loss: nan - loss: nan - reconstruction_loss: nan

InvalidArgumentError: Graph execution error:

Detected at node 'IteratorGetNext' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "/nix/store/1hzi3scdwn996mwyf66vvww6j0p9yj6p-python3-3.11.10-env/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "/nix/store/1hzi3scdwn996mwyf66vvww6j0p9yj6p-python3-3.11.10-env/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
      app.start()
    File "/nix/store/1hzi3scdwn996mwyf66vvww6j0p9yj6p-python3-3.11.10-env/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start
      self.io_loop.start()
    File "/nix/store/1hzi3scdwn996mwyf66vvww6j0p9yj6p-python3-3.11.10-env/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 205, in start
      self.asyncio_loop.run_forever()
    File "/nix/store/0bygl9k94j0xkxq7cmxnpqdq